<a href="https://colab.research.google.com/github/duatama2004/Komputasi_pararel/blob/main/proses%20ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok


In [ ]:
from pyngrok import ngrok

# Jalankan ngrok untuk MySQL (port 3306)
mysql_tunnel = ngrok.connect(3306, "tcp")
print("Ngrok URL:", mysql_tunnel.public_url)


ERROR:pyngrok.process.ngrok:t=2025-01-16T15:05:24+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-01-16T15:05:24+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-01-16T15:05:24+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
pip install pymysql


In [ ]:
from flask import Flask, request, render_template, jsonify
from flask_ngrok import run_with_ngrok
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pymysql

# Inisialisasi Flask
app = Flask(__name__)
run_with_ngrok(app)

# Load model CNN
from google.colab import drive
drive.mount('/content/drive')
MODEL_PATH = '/content/drive/MyDrive/CNN/model/Tomatoo_model.h5'
model = load_model(MODEL_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pymysql

# Koneksi ke MySQL
db = pymysql.connect(
    host='127.0.0.1',      # Ganti dengan host MySQL Anda (127.0.0.1 untuk lokal)
    user='root',           # Ganti dengan username MySQL Anda
    password='',           # Ganti dengan password MySQL Anda
    database='corn_disease'  # Ganti dengan nama database
)

cursor = db.cursor()
print("Koneksi berhasil!")


OperationalError: (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")

In [ ]:
from flask import Flask, request, render_template, jsonify
from flask_ngrok import run_with_ngrok
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pymysql

# Inisialisasi Flask
app = Flask(__name__)
run_with_ngrok(app)

# Load model CNN
from google.colab import drive
drive.mount('/content/drive')
MODEL_PATH = '/content/drive/MyDrive/CNN/model/Tomatoo_model.h5'
model = load_model(MODEL_PATH)

# MySQL Connection
db = pymysql.connect(
    host='127.0.0.1',  # Ganti dengan host MySQL Anda
    user='root',    # Ganti dengan username MySQL Anda
    password='',# Ganti dengan password MySQL Anda
    database='corn_disease'  # Nama database
)
cursor = db.cursor()

# Define class labels
CLASS_LABELS = ['Sehat', 'Penyakit A', 'Penyakit B']  # Sesuaikan dengan jenis penyakit

@app.route('/')
def home():
    return render_template('/content/drive/My Drive/CNN/templates/html.html')

@app.route('/classify', methods=['POST'])
def classify_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image file provided'}), 400

    file = request.files['image']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    # Preprocess image
    img = load_img(file, target_size=(128, 128))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict using the model
    predictions = model.predict(img_array)
    class_index = np.argmax(predictions)
    confidence = float(np.max(predictions))

    # Save to MySQL database
    sql = "INSERT INTO classifications (image_name, class_index, confidence) VALUES (%s, %s, %s)"
    values = (file.filename, class_index, confidence)
    cursor.execute(sql, values)
    db.commit()

    # Response
    return jsonify({
        'class_name': CLASS_LABELS[class_index],
        'confidence': confidence
    })

if __name__ == '__main__':
    app.run()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


OperationalError: (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")